<a href="https://colab.research.google.com/github/seifahmed2004/Information-retrieval-project-/blob/main/IR_project_seif_ahmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-terrier --quiet
!pip install nltk --quiet
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
!pip install transformers
!pip install flair
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup

In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import *
from nltk.stem.porter import *
from flair.data import Sentence
from flair.embeddings import WordEmbeddings
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
import tensorflow_hub as hub
import gensim
from gensim.models import Word2Vec
pd.set_option('display.max_colwidth', 150)
import pyterrier as pt
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [6]:
data = pd.read_csv("/content/emotion_sentimen_dataset.csv")
data

,ID,text,Emotion
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate
1,1,im so full of life i feel appalled,neutral
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral
...,...,...,...
452290,452290,i feel he did that because she could no longer have kids and could spend every waking moment devoted to him,neutral
452291,452291,i say i m feeling generous so have three winners lisa laurie and teresa,neutral
452292,452292,i feel bad for my comment,neutral
452293,452293,i don t want to suffer or be in pain or feel humiliated or hide from embarrassment,neutral


In [8]:
nltk.download('stopwords')
print(stopwords.words('english'))
nltk.download('stopwords')
nltk.download('punkt')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
stop_words = set(stopwords.words('english'))

In [10]:
# Function to remove stopwords
def remove_stopwords(text):

    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words] #Lower is used to normalize al the words make them in lower case
    print('Tokens are:',tokens,'\n')
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'content' column
data['processed_text'] = data['text'].apply(remove_stopwords)

Streaming output truncated to the last 5000 lines.
Tokens are: ['i', 'feel', 'that', 'the', 'bonds', 'of', 'friendship', 'are', 'just', 'as', 'strong', 'as', 'familial', 'bonds'] 

Tokens are: ['im', 'feeling', 'annoyed', 'or', 'not', 'in', 'a', 'good'] 

Tokens are: ['i', 'feel', 'this', 'is', 'unfortunate', 'and', 'wrong', 'headed'] 

Tokens are: ['i', 'think', 'since', 'before', 'my', 'last', 'post', 'ive', 'been', 'feeling', 'irritated', 'to', 'the', 'point', 'where', 'stupid', 'shit', 'gets', 'to', 'me'] 

Tokens are: ['i', 'would', 'like', 'to', 'see', 'the', 'company', 'come', 'out', 'with', 'a', 'leash', 'that', 'the', 'smaller', 'breeds', 'can', 'wear', 'as', 'i', 'feel', 'it', 'would', 'be', 'a', 'popular', 'size'] 

Tokens are: ['i', 'am', 'feeling', 'really', 'adventurous', 'i', 'll', 'even', 'have', 'some', 'homemade', 'bread', 'to', 'coincide', 'with', 'it'] 

Tokens are: ['i', 'have', 'been', 'very', 'drawn', 'to', 'flap', 'bags', 'lately', 'they', 'allude', 'this', 'fee

In [12]:
print("df after processing:\n")
data

df after processing:



,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,seriously hate one subject death feel reluctant drop
1,1,im so full of life i feel appalled,neutral,im full life feel appalled
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feelings think afraid accept possibility might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive really angry r feel like idiot trusting first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspicious one outside like rapture happened something
...,...,...,...,...
452290,452290,i feel he did that because she could no longer have kids and could spend every waking moment devoted to him,neutral,feel could longer kids could spend every waking moment devoted
452291,452291,i say i m feeling generous so have three winners lisa laurie and teresa,neutral,say feeling generous three winners lisa laurie teresa
452292,452292,i feel bad for my comment,neutral,feel bad comment
452293,452293,i don t want to suffer or be in pain or feel humiliated or hide from embarrassment,neutral,want suffer pain feel humiliated hide embarrassment


preprocessing

In [13]:
stemmer = PorterStemmer()

In [14]:
def Steem_text(text):

    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

In [15]:
#a function to clean the documents
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text
data['processed_text'] = data['processed_text'].apply(clean)
data

,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,seriously hate one subject death feel reluctant drop
1,1,im so full of life i feel appalled,neutral,im full life feel appalled
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feelings think afraid accept possibility might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive really angry r feel like idiot trusting first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspicious one outside like rapture happened something
...,...,...,...,...
452290,452290,i feel he did that because she could no longer have kids and could spend every waking moment devoted to him,neutral,feel could longer kids could spend every waking moment devoted
452291,452291,i say i m feeling generous so have three winners lisa laurie and teresa,neutral,say feeling generous three winners lisa laurie teresa
452292,452292,i feel bad for my comment,neutral,feel bad comment
452293,452293,i don t want to suffer or be in pain or feel humiliated or hide from embarrassment,neutral,want suffer pain feel humiliated hide embarrassment


In [16]:
data['processed_text']= data['processed_text'].apply(Steem_text)
data

,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop
1,1,im so full of life i feel appalled,neutral,im full life feel appal
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth
...,...,...,...,...
452290,452290,i feel he did that because she could no longer have kids and could spend every waking moment devoted to him,neutral,feel could longer kid could spend everi wake moment devot
452291,452291,i say i m feeling generous so have three winners lisa laurie and teresa,neutral,say feel gener three winner lisa lauri teresa
452292,452292,i feel bad for my comment,neutral,feel bad comment
452293,452293,i don t want to suffer or be in pain or feel humiliated or hide from embarrassment,neutral,want suffer pain feel humili hide embarrass


indexing

In [17]:
data['docno'] = data['ID'].astype(str)
data

,ID,text,Emotion,processed_text,docno
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop,0
1,1,im so full of life i feel appalled,neutral,im full life feel appal,1
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make,2
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place,3
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth,4
...,...,...,...,...,...
452290,452290,i feel he did that because she could no longer have kids and could spend every waking moment devoted to him,neutral,feel could longer kid could spend everi wake moment devot,452290
452291,452291,i say i m feeling generous so have three winners lisa laurie and teresa,neutral,say feel gener three winner lisa lauri teresa,452291
452292,452292,i feel bad for my comment,neutral,feel bad comment,452292
452293,452293,i don t want to suffer or be in pain or feel humiliated or hide from embarrassment,neutral,want suffer pain feel humili hide embarrass,452293


In [18]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
index_ref = indexer.index(data["processed_text"], data["docno"])
print(index_ref.toString())

10:46:21.427 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (5965) - further warnings are suppressed
10:50:55.744 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 23 empty documents
./DatasetIndex/data.properties


In [19]:
#we will first load the index
index = pt.IndexFactory.of(index_ref)
#we will call getCollectionStatistics() to check the stats
print(index.getCollectionStatistics().toString())

Number of documents: 452295
Number of terms: 50306
Number of postings: 3670249
Number of fields: 0
Number of tokens: 3819130
Field names: []
Positions:   false



In [20]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
triplet -> term17152 Nt=1 TF=1 maxTF=2147483647 @{0 5876466 6} 
tripletsplu -> term43016 Nt=1 TF=1 maxTF=2147483647 @{0 5876470 6} 
tripli -> term26550 Nt=2 TF=2 maxTF=2147483647 @{0 5876475 4} 
triplic -> term50215 Nt=1 TF=1 maxTF=2147483647 @{0 5876484 4} 
tripll -> term8589 Nt=5 TF=5 maxTF=2147483647 @{0 5876489 2} 
tripod -> term20257 Nt=9 TF=9 maxTF=2147483647 @{0 5876509 4} 
tripoli -> term31484 Nt=3 TF=3 maxTF=2147483647 @{0 5876542 4} 
trippel -> term20499 Nt=1 TF=1 maxTF=2147483647 @{0 5876555 2} 
tripper -> term28673 Nt=3 TF=3 maxTF=2147483647 @{0 5876559 4} 
trippi -> term31258 Nt=4 TF=4 maxTF=2147483647 @{0 5876573 0} 
trippin -> term40775 Nt=1 TF=1 maxTF=2147483647 @{0 5876589 6} 
trippl -> term25335 Nt=2 TF=2 maxTF=2147483647 @{0 5876594 4} 
triptr -> term8202 Nt=3 TF=3 maxTF=2147483647 @{0 5876602 2} 
triptych -> term8765 Nt=3 TF=3 maxTF=2147483647 @{0 5876614 2} 
trisomi -> term39834 Nt=1 TF=1 maxTF=2147483647 @{0 58766

In [21]:
index.getLexicon()["result"].getDocumentFrequency()

1329

In [22]:
x = index.getDirectIndex()
y = index.getDocumentIndex()
z = index.getLexicon()
a = 0
for posting in x.getPostings(y.getDocumentEntry(a)):
    termid = posting.getId()
    B = z.getLexiconEntry(termid)
    print("%s with frequency %d" % (B.getKey(),posting.getFrequency()))

hate with frequency 1
reluct with frequency 1
subject with frequency 1
death with frequency 1
drop with frequency 1
feel with frequency 1


Query processing

In [23]:
def preprocess(sentence):
    sentence = remove_stopwords(sentence)
    sentence = clean(sentence)
    sentence = Steem_text(sentence)
    return sentence

In [24]:
# Preprocess user query
query = "fun"
query = preprocess(query)

Tokens are: ['fun'] 



In [25]:
# Set up retrieval model with TF-IDF as weighting model
tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)

# Search for relevant documents using the retrieval model
results = tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,295451,295451,0,6.878604,fun
1,1,292699,292699,1,6.412099,fun
2,1,6419,6419,2,6.029267,fun
3,1,113446,113446,3,6.029267,fun
4,1,154301,154301,4,6.029267,fun
5,1,172715,172715,5,6.029267,fun
6,1,186026,186026,6,6.029267,fun
7,1,262359,262359,7,6.029267,fun
8,1,275863,275863,8,6.029267,fun
9,1,276118,276118,9,6.029267,fun


In [26]:
# Retrieve top relevant documents based on the query
top_answer = data[data["docno"].isin(['6419', '48097'])]['processed_text']
top_answer

6419                                                                     fun feel passion ever
48097    go back energi want feel take bodi amaz fun thought fun solut fun activ movement come
Name: processed_text, dtype: object

In [27]:
# Calculate frequency of the term 'empty' in the top related documents
freq_of_items = []
for i in range(len(top_answer)):
    tweet = top_answer.iloc[i].split()
    counter = 0
    for word_in_tweet in tweet:
        if word_in_tweet == 'fun':
            counter += 1
    freq_of_items.append(counter)
print('The frequency of the item in the top related docs are:', freq_of_items)

The frequency of the item in the top related docs are: [1, 3]


In [28]:
index = pt.IndexFactory.of(index_ref)

In [29]:
# Define our retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
results = bm25.search(query)
results

,qid,docid,docno,rank,score,query
0,1,295451,295451,0,12.594158,fun
1,1,292699,292699,1,11.740025,fun
2,1,6419,6419,2,11.039091,fun
3,1,113446,113446,3,11.039091,fun
4,1,154301,154301,4,11.039091,fun
5,1,172715,172715,5,11.039091,fun
6,1,186026,186026,6,11.039091,fun
7,1,262359,262359,7,11.039091,fun
8,1,275863,275863,8,11.039091,fun
9,1,276118,276118,9,11.039091,fun


In [30]:
data[['text']][data['docno'].isin(results['docno'].loc[0:4].tolist())]

,text
6419,i have fun i feel as passionate about it as ever
113446,i feel so blessed and got a lot of fun things
154301,i feel so sociable and fun
292699,i don t object to the fun i ve been having or the extent of the fun but more of the feeling of needing to do all of those things when unhappy
295451,i feel pressured to do fun things or because i am going through the motions of what a fun involved mom should do but because i am just having fun ...


In [31]:
rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)

rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off carefr^0.020895522 bless^0.041791044 fun^0.805970192 need^0.008955223 sociabl^0.020895522 selfish^0.020895522 passion^0.041791044 idiot^0.020895522 object^0.008955223 unhappi^0.008955223'

In [32]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

carefr^0.020895522
bless^0.041791044
fun^0.805970192
need^0.008955223
sociabl^0.020895522
selfish^0.020895522
passion^0.041791044
idiot^0.020895522
object^0.008955223
unhappi^0.008955223

fun


Query Expansion:

In [33]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))


#Let's check the tweets text for the top 5 retrieved tweets
data[['text']][data['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1    score_1  docid_2    score_2
0   295451  12.594158   295451  12.594158
1   292699  11.740025   292699  12.057847
2     6419  11.039091     6419  11.592460
3   113446  11.039091   262359  11.592460
4   154301  11.039091   423228  11.592460


,text
6419,i have fun i feel as passionate about it as ever
113446,i feel so blessed and got a lot of fun things
262359,i have fun i feel as passionate about it as ever
292699,i don t object to the fun i ve been having or the extent of the fun but more of the feeling of needing to do all of those things when unhappy
295451,i feel pressured to do fun things or because i am going through the motions of what a fun involved mom should do but because i am just having fun ...
423228,i have fun i feel as passionate about it as ever


In [34]:
!pip install flask_ngrok
!pip install pyngrok

In [35]:
df2 = data.head(50)

df2 = df2.to_dict()

df2

{'ID': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49},
 'text': {0: 'i seriously hate one subject to death but now i feel reluctant to drop it',
  1: 'im so full of life i feel appalled',
  2: 'i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it',
  3: 'ive been really angry with r and i feel like an idiot for trusting him in the first place',
  4: 'i feel suspicious if there is no one outside like the rapture has happened or something',
  5: 'i feel jealous becasue i wanted that kin

In [36]:
def sui(df2 , que):
 i = 0

 quer = preprocess(que)

 docs_id = []

 for key, value in df2.items():
   if key == 'processed_text':
         val = value.values()
         for doc in val:
           terms = doc.split()
           for term in terms:
             if term == quer and i not in docs_id:
               docs_id.append(f'''Document number {i} -----> \n{data["text"][i]}''')
           i = i + 1
 return docs_id

In [37]:
query2 = "like"
X = sui(df2 , query2)
X

Tokens are: ['like'] 



['Document number 3 -----> \nive been really angry with r and i feel like an idiot for trusting him in the first place',
 'Document number 4 -----> \ni feel suspicious if there is no one outside like the rapture has happened or something',
 'Document number 9 -----> \ni feel like i ve regained another vital part of my life which is living',
 'Document number 10 -----> \ni feel a bit like franz liebkind in the producers not many people know it but the fuhrer was a terrific dancer',
 'Document number 14 -----> \ni just try not to talk to anyone when i feel irritable like that',
 'Document number 15 -----> \ni feel like they hated me since then',
 'Document number 19 -----> \ni feel like i have to redeem myself even though i think they realized why i was distraught and were ok with it',
 'Document number 33 -----> \ni end up getting unwanted attention from boys i want little to do with or ill be sort of starting something with a boy then find myself flirtiing with others in his presence o

In [39]:
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-uncased"

bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [40]:
imdb_dataset_url = "https://raw.githubusercontent.com/LearnDataSci/articles/master/Python%20Pandas%20Tutorial%20A%20Complete%20Introduction%20for%20Beginners/IMDB-Movie-Data.csv"

reviews = data
reviews.head()

,ID,text,Emotion,processed_text,docno
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop,0
1,1,im so full of life i feel appalled,neutral,im full life feel appal,1
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make,2
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place,3
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth,4


In [46]:
from tqdm import tqdm

def encode(text, max_length=32):
    return bert_tokenizer.encode_plus(
                  text,                      # Sentence to encode.
                  add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                  truncation=True,
                  max_length = max_length,           # Pad & truncate all sentences.
                  padding="max_length",
                  return_attention_mask = True,   # Construct attention mask
                  return_tensors = 'pt',     # Return pytorch tensors.
    )

tokenized_reviews = []
for tweet in tqdm(reviews["Emotion"].values, desc="Tokenizing ..."):
    tokenized_reviews.append(encode(tweet, max_length=32))

Tokenizing ...: 100%|██████████| 452295/452295 [01:19<00:00, 5714.08it/s]


In [47]:
bert_model = bert_model.cpu()

In [48]:
import torch

device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
bert_model = bert_model.to(device)
bert_model.device

device(type='cpu')

In [50]:
input_ids = tokenized_reviews[0]["input_ids"].to(device)
attention_mask = tokenized_reviews[0]["attention_mask"].to(device)
output = bert_model(input_ids=input_ids, attention_mask=attention_mask)


In [51]:
text_king = "king"
text_man = "man"
text_apple = "apple"

# Function to convert text to embeddings
def get_embeddings(text):
  # Encode text and computer input_ids and attention mask
  tokens = encode(text)
  input_ids = tokens["input_ids"].to(device)
  attention_mask = tokens["attention_mask"].to(device)
  # Pass input_ids and attention mask to model
  output = bert_model(input_ids=input_ids, attention_mask=attention_mask)
  return output

embed_king = get_embeddings(text_king)
embed_man = get_embeddings(text_man)
embed_apple = get_embeddings(text_apple)

In [52]:
embed_king

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1318,  0.1997,  0.0593,  ..., -0.1656, -0.0187, -0.0151],
         [-0.1006, -0.0524,  0.0772,  ..., -0.2972, -0.1973, -0.6630],
         [ 0.8228,  0.2570, -0.2939,  ..., -0.0014, -0.8282, -0.2063],
         ...,
         [-0.4145, -0.0221,  0.6602,  ..., -0.0110,  0.0749, -0.2281],
         [-0.2734,  0.0148,  0.7546,  ..., -0.0276,  0.0481, -0.2719],
         [-0.6719, -0.0510,  0.0872,  ...,  0.0179,  0.0716, -0.2357]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.7962, -0.2311,  0.4424,  0.6373, -0.3603, -0.1046,  0.8782,  0.1833,
          0.2154, -0.9996,  0.1175,  0.2204,  0.9610, -0.2259,  0.8968, -0.3678,
         -0.1455, -0.4299,  0.3994, -0.7417,  0.5365,  0.8699,  0.6029,  0.1351,
          0.3400,  0.2738, -0.5180,  0.8717,  0.9329,  0.6474, -0.6485,  0.1758,
         -0.9658, -0.2114,  0.3730, -0.9662,  0.1247, -0.7125, -0.0458,  0.0131,
         -0.8082,  0.2329,  0.98

In [53]:
# Embedding of the word "king" at index 1 (since index 0 is reserved for [CLS] token)
embed_king[0][0][1]

tensor([-1.0063e-01, -5.2393e-02,  7.7211e-02, -3.3986e-01, -1.0123e-01,
        -3.0228e-02,  1.7702e-01, -1.2432e-01, -4.0533e-01, -1.0918e+00,
        -8.1277e-01,  1.5461e-01,  1.9359e-01,  4.6548e-01,  1.2872e-01,
        -5.5175e-02, -1.7733e-02, -6.9906e-02,  3.8718e-01,  1.1769e-01,
        -3.8392e-01,  1.5190e-01, -7.4895e-03, -9.0574e-02,  2.6472e-01,
         5.8122e-01, -1.8173e-01,  1.4358e-01,  8.3956e-02,  4.5901e-01,
         3.5349e-01, -6.7306e-01,  1.6623e-01,  2.6586e-01, -5.9197e-01,
        -4.3626e-01,  5.0111e-01, -2.3916e-01, -6.2190e-01, -1.3644e-01,
         4.4553e-01, -5.7102e-01,  7.1198e-01, -4.4230e-01,  2.3134e-01,
         2.2516e-01,  1.0018e-01,  1.6921e-01,  2.4917e-01,  2.6961e-01,
        -8.8482e-02,  2.2383e-01, -6.7094e-01,  2.2819e-01,  1.8186e-01,
         6.0945e-02,  5.2540e-01,  3.3595e-01, -5.2457e-01, -2.2025e-01,
         4.0992e-01, -8.3914e-01, -3.9757e-02, -5.4600e-01,  9.7653e-02,
         6.9954e-01,  1.1494e+00,  2.9574e-01, -1.0

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
# helper function
def compute_cosine_similarity(a, b):
    # Move array to CPU memory, detach from computational graph which requires_grad, convert to numpy & reshape
    a = a.cpu().detach().numpy().reshape(1, -1)
    b = b.cpu().detach().numpy().reshape(1, -1)
    cos_sim = cosine_similarity(a, b)
    return cos_sim

In [56]:
# Compare cosine similarity between (king, man, appple) embeddings

king_apple_similarity = compute_cosine_similarity(embed_king[0][0][1], embed_apple[0][0][1])
print(f"Similarity between King & Apple is {king_apple_similarity * 100}")

king_man_similarity = compute_cosine_similarity(embed_king[0][0][1], embed_man[0][0][1])
print(f"Similarity between King & Man is {king_man_similarity * 100}")

man_apple_similarity = compute_cosine_similarity(embed_man[0][0][1], embed_apple[0][0][1])
print(f"Similarity between Man & Apple is {man_apple_similarity * 100}")

Similarity between King & Apple is [[66.7224]]
Similarity between King & Man is [[74.478806]]
Similarity between Man & Apple is [[65.900024]]


In [57]:
king_queen_similarity = compute_cosine_similarity(get_embeddings("queen")[0][0][1], embed_king[0][0][1])
print(f"Similarity between King & Queen is {king_queen_similarity * 100}")

Similarity between King & Queen is [[86.438576]]


UI

In [59]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

https://njblnt09ifi-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok

# Assuming you've already defined the sui function and imported necessary modules

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Unique Search Engine</title>
    <style>
        body {
            background-color: #f1c232;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }

        .header {
            background-color: #f1c232;
            color: black;
            padding: 20px 0;
            text-align: center;
        }

        .container {
            text-align: center;
            margin-top: 50px;
        }

        h1 {
            margin: 0;
            padding: 10px 0;
        }

        #searchInput {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 50px;
            margin-bottom: 10px;
            width: 300px;
            box-sizing: border-box;
            transition: border-color 0.5s;
        }

        #searchInput:focus {
            border-color: #403072;
        }

        button {
            padding: 10px 20px;
            background-color: #ce7e00;
            color: white;
            border: none;
            border-radius: 20px;
            cursor: pointer;
            transition: background-color 0.5s;
        }

        button:hover {
            background-color: #7d73af;
        }

        #searchResult {
            text-align: left;
            color: black;
            margin: 20px auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 10px;
            max-width: 600px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }

        #searchResult h2 {
            border-bottom: 2px solid #ce7e00;
            padding-bottom: 10px;
            margin-bottom: 20px;
        }

        #searchResult p {
            margin: 10px 0;
            font-size: 16px;
        }

        .footer {
            background-color: #f1c232;
            color: black;
            padding: 20px 0;
            text-align: center;
            position: fixed;
            bottom: 0;
            width: 100%;
        }
    </style>
</head>
<body>

<div class="header">
    <h1>🔍 Unique Search Engine 🔍</h1>
</div>

<div class="container">
    <input type="text" id="searchInput" placeholder="Enter your query...">
    <button onclick="search()">Search</button>
</div>

<div id="searchResult"></div>

<div class="footer">
    <p>Made with ❤️ by Seif Ahmed</p>
</div>

<script>
    function search() {
        var searchTerm = document.getElementById("searchInput").value;
        fetch('/search', {
            method: 'POST',
            body: JSON.stringify({ query: searchTerm }),
            headers:{
                'Content-Type': 'application/json'
            }
        })
        .then(response => response.json())
        .then(data => {
            var resultDiv = document.getElementById("searchResult");
            resultDiv.innerHTML = ""; // Clear previous results
            if (data.results.length === 0) {
                resultDiv.innerHTML = "<h2>No documents found 😔</h2>";
            } else {
                resultDiv.innerHTML = "<h2>Search Results 📚</h2>";
                data.results.forEach(doc => {
                    resultDiv.innerHTML += "<p>" + doc + "</p>";
                });
            }
        })
        .catch(error => {
            console.error('Error occurred during fetch:', error);
        });
    }
</script>

</body>
</html>
    """

@app.route("/search", methods=['POST'])
def search():
    query = request.json['query']
    print("Received query:", query)  # Debug: Check if Flask receives the query
    results = sui(df2, query)
    print("Search results:", results)  # Debug: Check if sui function returns results
    return {'results': results}

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-23:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 715, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packag

In [61]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

df= vaswani_dataset.get_topics()

df['docno'] = df.index

# Rename column 'A' to 'X'
df = df.rename(columns={'query': 'Text'})

qrels = vaswani_dataset.get_qrels()

qrels['docno']=qrels['docno'].astype(str)

df

query-text.trec:   0%|          | 0.00/3.05k [00:00<?, ?iB/s]

qrels:   0%|          | 0.00/6.63k [00:00<?, ?iB/s]

,qid,Text,docno
0,1,measurement of dielectric constant of liquids by the use of microwave techniques,0
1,2,mathematical analysis and design details of waveguide fed microwave radiations,1
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics,2
3,4,systems of data coding for information transfer,3
4,5,use of programs in engineering testing of computers,4
...,...,...,...
88,89,tunnel diode construction and its electrical characteristics explained,88
89,90,electronic density of states at the surface of a semiconductor compared with that at depth,89
90,91,resistivity of metallic thin films related to surface roughness,90
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields,91


In [62]:
indexref2 = vaswani_dataset.get_index()
index2 = pt.IndexFactory.of(indexref2)

print(index2.getCollectionStatistics().toString())

data.direct.bf:   0%|          | 0.00/388k [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/234k [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/362k [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/682k [00:00<?, ?iB/s]

data.lexicon.fsomaphash:   0%|          | 0.00/777 [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/89.3k [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/224k [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.29k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/619 [00:00<?, ?iB/s]

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 1
Number of tokens: 271581
Field names: [text]
Positions:   false



In [63]:
retr = pt.BatchRetrieve(index2, controls = {"wmodel": "TF_IDF"})

res = retr.search("mathematical")
res

,qid,docid,docno,rank,score,query
0,1,4746,4747,0,5.168347,mathematical
1,1,7399,7400,1,5.036916,mathematical
2,1,5629,5630,2,4.912003,mathematical
3,1,7997,7998,3,4.912003,mathematical
4,1,4546,4547,4,4.679886,mathematical
...,...,...,...,...,...,...
147,1,3484,3485,147,1.828498,mathematical
148,1,7283,7284,148,1.747822,mathematical
149,1,6714,6715,149,1.702745,mathematical
150,1,8622,8623,150,1.606095,mathematical


In [64]:
eval = pt.Evaluate(res,qrels)
eval

{'map': 4.7960250544348844e-06, 'ndcg': 0.00022891881462746983}